In [456]:
import torch
torch.cuda.is_available()
print(torch.zeros(1).cuda())

import torch_geometric
import torch_geometric.nn as pyg_nn
from torch_sparse import SparseTensor, matmul
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv
print(torch_geometric.__version__)

import torch_scatter
import torch.nn as nn
import torch.nn.functional as F

import torch_geometric.utils as pyg_utils

from torch import Tensor
from typing import Union, Tuple, Optional
from torch_geometric.typing import (OptPairTensor, Adj, Size, NoneType, OptTensor)

from torch.nn import Parameter, Linear
from torch_sparse import SparseTensor, set_diag
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops, softmax

import networkx as nx
from deepsnap.hetero_graph import HeteroGraph
import copy
from copy import deepcopy
import pickle

import deepsnap
from deepsnap.graph import Graph
from deepsnap.batch import Batch
from deepsnap.dataset import GraphDataset
from deepsnap.hetero_gnn import forward_op
from deepsnap.hetero_graph import HeteroGraph
from sklearn.metrics import f1_score, roc_auc_score

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import json
from pathlib import Path as Data_Path
from os import listdir
from os.path import isfile, join
from itertools import combinations
from tqdm.notebook import tqdm

import graph_tool.all as gt
import json
print("graph-tool version: {}".format(gt.__version__.split(' ')[0]))

tensor([0.], device='cuda:0')
2.1.0
graph-tool version: 2.46


In [457]:
import pickle

with open("../graph_pt_100.pickle", "rb") as f:
    final_graph = pickle.load(f)

In [458]:
print("Number of vertices:", final_graph.num_vertices())
print("Number of edges:", final_graph.num_edges())

Number of vertices: 3644438
Number of edges: 6746550


In [459]:
print('without artists as nodes:')
print("Number of vertices:", final_graph.num_vertices())
print("Number of edges:", final_graph.num_edges())

without artists as nodes:
Number of vertices: 3644438
Number of edges: 6746550


In [460]:
# count_pl = sum(1 for v in final_graph.vertices() if final_graph.vp.pl_type[v] == "playlist")
# print(f'num_playlists: {count_pl}')
# count_tr = sum(1 for v in final_graph.vertices() if final_graph.vp.tr_type[v] == "track")
# print(f'num_tracks: {count_tr}')
# count_ar = sum(1 for v in final_graph.vertices() if final_graph.vp.ar_type[v] == "artist")
# print(f'num_artist: {count_ar}')

In [461]:
print('without artists as nodes:')
count_pl = sum(1 for v in final_graph.vertices() if final_graph.vp.pl_type[v] == "playlist")
print(f'num_playlists: {count_pl}')
count_tr = sum(1 for v in final_graph.vertices() if final_graph.vp.tr_type[v] == "track")
print(f'num_tracks: {count_tr}')
count_ar = sum(1 for v in final_graph.vertices() if final_graph.vp.ar_type[v] == "artist")
print(f'num_artist: {count_ar}')

without artists as nodes:
num_playlists: 102000
num_tracks: 3542438
num_artist: 0


In [462]:
### Removal of duplicate nodes from pickle file


# prop_name = "pl_type"

# # Create a dictionary of property values to vertex IDs
# vertex_dict = {}
# for v in final_graph.vertices():
#     value = final_graph.vp[prop_name][v]
#     if value not in vertex_dict:
#         vertex_dict[value] = [int(v)]
#     else:
#         vertex_dict[value].append(int(v))

# # Remove duplicate nodes
# for value, ids in vertex_dict.items():
#     if len(ids) > 1:
#         print(value,ids)
#         # # Merge the duplicate nodes into the first node
#         # first_id = ids[0]
#         # for other_id in ids[1:]:
#         #     final_graph.vertex(first_id).out_edges()[:] = gt.find_edge(final_graph, final_graph.vertex(first_id), final_graph.vertex(other_id))
#         #     final_graph.vertex(first_id).out_edges()[:] = gt.find_edge(final_graph, final_graph.vertex(other_id), final_graph.vertex(first_id))
#         #     final_graph.remove_vertex(final_graph.vertex(other_id))


# print("Number of vertices:", final_graph.num_vertices())
# print("Number of edges:", final_graph.num_edges())

### Largest component

#### Graph-tool

In [463]:
largest_comp = gt.extract_largest_component(final_graph)
# largest_comp = gt.GraphView(final_graph, vfilt = gt.label_largest_component(final_graph))

In [464]:
print('no artists:')
print("Number of vertices:", largest_comp.num_vertices()) 
print("Number of edges:", largest_comp.num_edges())

no artists:
Number of vertices: 38565
Number of edges: 6746550


In [465]:
print('no artists stats:')
print('----------------')
count_pl = sum(1 for v in largest_comp.vertices() if largest_comp.vp.pl_type[v] == "playlist")
print(f'num_playlists: {count_pl}')
count_tr = sum(1 for v in largest_comp.vertices() if largest_comp.vp.tr_type[v] == "track")
print(f'num_tracks: {count_tr}')
count_ar = sum(1 for v in largest_comp.vertices() if largest_comp.vp.ar_type[v] == "artist")
print(f'num_artist: {count_ar}')

no artists stats:
----------------
num_playlists: 2999
num_tracks: 35566
num_artist: 0


In [466]:
# #### Networkx
# nx_graph = nx.Graph()
# for node in final_graph.vertices():
#     nx_graph.add_node(int(node))
# for edge in final_graph.edges():
#     nx_graph.add_edge(int(edge.source()), int(edge.target()))

# from networkx.algorithms.components import is_connected
# is_connected(nx_graph)

# largest_cc = max(nx.connected_components(nx_graph), key=len)
# nx_largest_comp = nx.Graph(nx_graph.subgraph(largest_cc))
# print('Num nodes:', nx_largest_comp.number_of_nodes(), '. Num edges:', nx_largest_comp.number_of_edges())

### N-hop neighbourhood

In [467]:
pass

### Deepsnap

In [469]:
print(g_nx.nodes[1000])

{'name': 'Awesome Playlist', 'uri': ''}


In [ ]:
print(g_nx.edges(560))
print(largest_comp.vp.pl_name[10])

[(560, 23579), (560, 23580), (560, 23581), (560, 23582), (560, 23583), (560, 23584), (560, 23585), (560, 23586), (560, 23587), (560, 23588), (560, 23589), (560, 23590), (560, 23591), (560, 23592), (560, 23593), (560, 23594), (560, 23595), (560, 23596), (560, 23597), (560, 23598), (560, 23599), (560, 23600), (560, 24627), (560, 24628), (560, 24629), (560, 24630), (560, 24631), (560, 24632), (560, 24633), (560, 24634), (560, 24635), (560, 24636), (560, 24637), (560, 24638), (560, 24639), (560, 24640), (560, 24641), (560, 24642), (560, 24643), (560, 24644), (560, 24645), (560, 24646), (560, 24647), (560, 24648), (560, 24649), (560, 24650), (560, 24651), (560, 24652), (560, 24653), (560, 24654), (560, 24655), (560, 24656), (560, 24657), (560, 24658), (560, 24659), (560, 24660), (560, 24661), (560, 24662), (560, 24663), (560, 24664), (560, 24665), (560, 24666), (560, 24667), (560, 24668), (560, 24669), (560, 24670), (560, 24671), (560, 24672), (560, 24673), (560, 24674), (560, 24675), (560,

In [470]:
print("Number of vertices:", largest_comp.num_vertices()) 
print("Number of edges:", largest_comp.num_edges())

Number of vertices: 38565
Number of edges: 6746550


In [471]:
task = 'link_pred'
dataset = GraphDataset([ds_graph], task=task, edge_train_mode='disjoint')

dataset_train, dataset_val, dataset_test = dataset.split(transductive=True, split_ratio=[0.8, 0.1, 0.1])

# dataset_train[0].to('cuda')
# dataset_val[0].to('cuda')
# dataset_test[0].to('cuda')

# dataset_train.to('cuda:0')
# dataset_val.to('cuda:0')
# dataset_test.to('cuda:0')

print(type(dataset_train))
print(dataset_train[0])
print(type(dataset_train[0]))

num_train_edges = dataset_train[0].edge_label_index.shape[1]
num_val_edges = dataset_val[0].edge_label_index.shape[1]
num_test_edges = dataset_test[0].edge_label_index.shape[1]

print("Train set has {} supervision (positive) edges".format(num_train_edges // 2))
print("Validation set has {} supervision (positive) edges".format(num_val_edges // 2))
print("Test set has {} supervision (positive) edges".format(num_test_edges // 2))

print("Train set has {} message passing edges".format(dataset_train[0].edge_index.shape[1]))
print("Validation set has {} message passing edges".format(dataset_val[0].edge_index.shape[1]))
print("Test set has {} message passing edges".format(dataset_test[0].edge_index.shape[1]))

<class 'deepsnap.dataset.GraphDataset'>
Graph(G=[], edge_index=[2, 2320892], edge_label=[1160448], edge_label_index=[2, 1160448], name=[38565], negative_label_val=[1], node_label_index=[38565], uri=[38565])
<class 'deepsnap.graph.Graph'>
Train set has 580224 supervision (positive) edges
Validation set has 362638 supervision (positive) edges
Test set has 362642 supervision (positive) edges
Train set has 2320892 message passing edges
Validation set has 2901116 message passing edges
Test set has 3263754 message passing edges


In [472]:
# pickle.dump(dataset_train, open('./graphs/train.graph', 'wb'))
# pickle.dump(dataset_val, open('./graphs/val.graph', 'wb'))
# pickle.dump(dataset_test, open('./graphs/test.graph', 'wb'))

In [473]:
# dataset_train = pickle.load(open('./graphs/train.graph', 'rb'))
# dataset_val = pickle.load(open('./graphs/val.graph', 'rb'))
# dataset_test = pickle.load(open('./graphs/test.graph', 'rb'))

In [474]:
class LightGCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels, normalize = True,
                 bias = False, **kwargs):  
        super(LightGCNConv, self).__init__(**kwargs)

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.normalize = normalize

    def forward(self, x, edge_index, size = None):
        out = self.propagate(edge_index, x=(x, x))
        return out

    def message(self, x_j):
        out = x_j
        return out

    def aggregate(self, inputs, index, dim_size = None):
        node_dim = self.node_dim
        out = torch_scatter.scatter(inputs, index, dim=node_dim, reduce='mean')
        return out

In [475]:
class LightGCN(torch.nn.Module):
    def __init__(self, train_data, num_layers, emb_size=16, initialize_with_words=False):
        super(LightGCN, self).__init__()
        self.convs = nn.ModuleList()
        assert (num_layers >= 1), 'Number of layers is not >=1'
        for l in range(num_layers):
            self.convs.append(LightGCNConv(input_dim, input_dim))

        # Initialize using custom embeddings if provided
        num_nodes = train_data.node_label_index.size()[0]
        self.embeddings = nn.Embedding(num_nodes, emb_size)
        if initialize_with_words:
            self.embeddings.weight.data.copy_(train_datanode_features)
        
        self.loss_fn = nn.BCELoss()
        self.num_layers = num_layers
        self.emb_size = emb_size
        self.num_modes = num_nodes

    def forward(self, data):
        edge_index, edge_label_index, node_label_index = data.edge_index, data.edge_label_index, data.node_label_index
        layer_embeddings = []
        
        x = self.embeddings(node_label_index)
        mean_layer = x

        # We take an average of ever layer's node embeddings
        for i in range(self.num_layers):
            x = self.convs[i](x, edge_index)
            # print("x shape",x.shape)
            # print("mean_layer shape",mean_layer.shape)
            mean_layer += x

        mean_layer /= 4

        # Prediction head is simply dot product
        nodes_first = torch.index_select(x, 0, edge_label_index[0,:].long())
        nodes_second = torch.index_select(x, 0, edge_label_index[1,:].long())

        # Since we don't want a rank output, we create a sigmoid of the dot product
        out = torch.sum(nodes_first * nodes_second, dim=-1) # FOR RANKING
        pred = torch.sigmoid(out)

        return torch.flatten(pred)

    def loss(self, pred, label):
        return self.loss_fn(pred, label)

In [476]:
import warnings
warnings.filterwarnings('ignore')

In [477]:
print(type(dataset_train))

<class 'deepsnap.dataset.GraphDataset'>


In [478]:
args = {
    'device' : 'cuda' if torch.cuda.is_available() else 'cpu',
    'num_layers' : 3,
    'emb_size' : 32,
    'weight_decay': 1e-5,
    'lr': 0.01,
    'epochs': 600
}

datasets = {
    'train': dataset_train[0],
    'val': dataset_val[0],
    'test': dataset_test[0]
}
            
input_dim = datasets['train'].num_node_features
print(input_dim, args)

0 {'device': 'cuda', 'num_layers': 3, 'emb_size': 32, 'weight_decay': 1e-05, 'lr': 0.01, 'epochs': 600}


In [479]:
# datasets['train'].to(args['device'])
# datasets['val'].to(args['device'])
# datasets['test'].to(args['device'])

In [480]:
losses = []

def train(model, optimizer, args):
    val_max = 0
    best_model = model

    for epoch in range(1, args['epochs'] + 1):
        datasets['train'].to(args["device"])
        model.train()
        optimizer.zero_grad()
        pred = model(datasets['train'])
        loss = model.loss(pred, datasets['train'].edge_label.type(pred.dtype))

        loss.backward()
        optimizer.step()

        log = 'Epoch: {:03d}, Train: {:.4f}, Val: {:.4f}, Test: {:.4f}, Loss: {:.5f}, Val Loss: {:.5f}'
        score_train, train_loss = test(model, 'train', args)
        score_val, val_loss = test(model, 'val', args)
        score_test, test_loss = test(model, 'test', args)

        losses.append((train_loss, val_loss))

        print(log.format(epoch, score_train, score_val, score_test, train_loss, val_loss))
        if val_max < score_val:
            val_max = score_val
            best_model = copy.deepcopy(model)

    return best_model

def test(model, mode, args):
    model.eval()
    score = 0
    loss_score = 0

    data = datasets[mode]
    data.to(args["device"])

    pred = model(data)
    loss = model.loss(pred, data.edge_label.type(pred.dtype))
    score += roc_auc_score(data.edge_label.flatten().cpu().numpy(), pred.flatten().data.cpu().numpy())
    loss_score += loss.item()

    return score, loss_score

In [481]:
model = LightGCN(datasets['train'], args['num_layers'], emb_size=args['emb_size']).to(args['device'])
optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'], weight_decay=args['weight_decay'])

best_model = train(model, optimizer, args)
log = "Train: {:.4f}, Val: {:.4f}, Test: {:.4f}, Train Loss: {:.5f}, Val Loss: {:.5f}, Test Loss: {:.5f}"
best_train_roc, train_loss = test(best_model, 'train', args)
best_val_roc, val_loss = test(best_model, 'val', args)
best_test_roc, test_loss = test(best_model, 'test', args)
print(log.format(best_train_roc, best_val_roc, best_test_roc, train_loss, val_loss, test_loss))

Epoch: 001, Train: 0.4867, Val: 0.4861, Test: 0.4831, Loss: 0.69831, Val Loss: 0.69832
Epoch: 002, Train: 0.5586, Val: 0.5585, Test: 0.5554, Loss: 0.69573, Val Loss: 0.69572
Epoch: 003, Train: 0.6083, Val: 0.6083, Test: 0.6056, Loss: 0.69394, Val Loss: 0.69392
Epoch: 004, Train: 0.6411, Val: 0.6409, Test: 0.6387, Loss: 0.69277, Val Loss: 0.69274
Epoch: 005, Train: 0.6619, Val: 0.6616, Test: 0.6597, Loss: 0.69202, Val Loss: 0.69198
Epoch: 006, Train: 0.6756, Val: 0.6750, Test: 0.6732, Loss: 0.69155, Val Loss: 0.69151
Epoch: 007, Train: 0.6853, Val: 0.6845, Test: 0.6827, Loss: 0.69124, Val Loss: 0.69119
Epoch: 008, Train: 0.6930, Val: 0.6919, Test: 0.6903, Loss: 0.69100, Val Loss: 0.69095
Epoch: 009, Train: 0.6999, Val: 0.6987, Test: 0.6970, Loss: 0.69078, Val Loss: 0.69073
Epoch: 010, Train: 0.7064, Val: 0.7052, Test: 0.7036, Loss: 0.69055, Val Loss: 0.69050
Epoch: 011, Train: 0.7128, Val: 0.7116, Test: 0.7100, Loss: 0.69030, Val Loss: 0.69025
Epoch: 012, Train: 0.7191, Val: 0.7179, Tes

In [482]:
#model_save
# filename = 'lgcn.pkl'
# torch.save(model, filename)

#datasets save
# with open('datasets_deepsnap_dict.pkl', 'wb') as f:
#     # pickle the dictionary and write it to the file
#     pickle.dump(datasets, f)

model = torch.load('lgcn.pkl')
with open('datasets_deepsnap_dict.pkl', 'rb') as f:
    # load the pickled dictionary from the file
    datasets = pickle.load(f)

In [483]:
def plot_learning_curve(losses, title):
    train_loss, val_loss = zip(*losses)
    steps = list(range(1, len(train_loss) + 1))
    
    min_val_loss = np.round(np.min(val_loss), 3)
    # train_list = [math.log10(x) for x in train_loss]
    # val_list = [math.log10(x) for x in val_loss]
    
    plt.figure(figsize=(16, 6))
    plt.plot(steps, train_loss, '-r', label='Training Loss')
    plt.plot(steps, val_loss, '-b', label='Validation Loss')
    plt.hlines(min_val_loss, 1, 300, colors='k', linestyles='dotted', label='Min Validation Loss: {}'.format(min_val_loss))

    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    # plt.ylim((0.58, 0.71))
    plt.title(title)
    plt.legend(loc='upper right')
    plt.title(title)

    return plt

In [484]:
plot_learning_curve(losses, 'test')

<module 'matplotlib.pyplot' from '/home/asd27/.conda/envs/cudatorch/lib/python3.11/site-packages/matplotlib/pyplot.py'>

In [485]:
data = datasets['test']
pred = model(data)

In [486]:
pred

tensor([0.7070, 0.6771, 0.6789,  ..., 0.4114, 0.4883, 0.4203], device='cuda:0',
       grad_fn=<SigmoidBackward0>)

In [487]:
len(pred)

725284

In [488]:
ds = datasets['test']
ds

Graph(G=[], edge_index=[2, 3263754], edge_label=[725284], edge_label_index=[2, 725284], name=[38565], negative_label_val=[1], node_label_index=[38565], uri=[38565])

In [489]:
ds.edge_label

tensor([1, 1, 1,  ..., 0, 0, 0], device='cuda:0')

In [490]:
type(ds.edge_label_index)

torch.Tensor

In [545]:
dict1 = {'preds':pred.to('cpu').detach().numpy(), 'edge_label': ds.edge_label.to('cpu').detach().numpy(), 'src_node': ds.edge_label_index[0].to('cpu').detach().numpy(), 'dest_node': ds.edge_label_index[1].to('cpu').detach().numpy()}
df = pd.DataFrame(dict1)
df.head()

preds  edge_label  src_node  dest_node
0  0.706988           1       888      33215
1  0.677142           1       796      29493
2  0.678916           1       484      20997
3  0.665643           1       567      22428
4  0.675998           1       560      22430

In [546]:
df_0 = df[df['edge_label'] == 0]
df_0

preds  edge_label  src_node  dest_node
362642  0.526130           0     27028      22087
362643  0.811317           0     32596      34830
362644  0.470571           0     12490      37364
362645  0.785511           0     12379      10949
362646  0.321483           0     22202      32728
...          ...         ...       ...        ...
725279  0.412675           0     23203      17151
725280  0.397621           0      8408      23257
725281  0.411401           0      7501      27248
725282  0.488291           0     30544          9
725283  0.420324           0     36239      18796

[362642 rows x 4 columns]

In [547]:
df_lessthan = df_0[df_0['preds'] > 0.5]
df_lessthan

preds  edge_label  src_node  dest_node
362642  0.526130           0     27028      22087
362643  0.811317           0     32596      34830
362645  0.785511           0     12379      10949
362647  0.510759           0      1574       7400
362648  0.512315           0     37935       2377
...          ...         ...       ...        ...
725267  0.605943           0     17330      21269
725268  0.694120           0     12287       9784
725270  0.557932           0     20264      15797
725273  0.872649           0     26852      26885
725276  0.564564           0      7257       3760

[101994 rows x 4 columns]

In [494]:
df_lessthan.iloc[0]['src_node']

27028.0

In [495]:
g_nx[0]

AtlasView({1000: {}, 1001: {}, 1002: {}, 1003: {}, 1004: {}, 1005: {}, 1006: {}, 1007: {}, 1008: {}, 1009: {}, 1010: {}, 1011: {}, 1012: {}, 1013: {}, 1014: {}, 1015: {}, 1016: {}, 1017: {}, 1018: {}, 1019: {}, 1020: {}, 1021: {}, 1022: {}, 1023: {}, 1024: {}, 1025: {}, 1026: {}, 1027: {}, 1028: {}, 1029: {}, 1030: {}, 1031: {}, 1032: {}, 1033: {}, 1034: {}, 1035: {}, 1036: {}, 1037: {}, 1038: {}, 1039: {}, 1040: {}, 1041: {}, 1042: {}, 1043: {}, 1044: {}, 1045: {}, 1046: {}, 1047: {}, 1048: {}, 1049: {}, 1050: {}, 1051: {}, 1052: {}, 1053: {}, 1054: {}, 1055: {}, 1056: {}, 1057: {}, 1058: {}, 1059: {}, 1060: {}, 1061: {}, 1062: {}, 1063: {}, 1064: {}, 1065: {}, 1066: {}, 1067: {}, 1068: {}, 1069: {}, 1070: {}, 1071: {}, 1072: {}, 1073: {}, 1074: {}, 1075: {}, 1076: {}, 1077: {}, 1078: {}, 1079: {}, 1080: {}, 1081: {}, 1082: {}, 1083: {}, 1084: {}, 1085: {}, 1086: {}, 1087: {}, 1088: {}, 1089: {}, 1090: {}, 1091: {}, 1092: {}, 1093: {}, 1094: {}, 1095: {}, 1096: {}, 1097: {}, 1098: {},

In [536]:
df_lessthan.iloc[0]["src_node"]
counter=0
neg=0
for x in g_nx.nodes:
    if(counter!=x):
        neg=neg+x-counter
        counter=x
    counter=counter+1
print(len(g_nx.nodes))
neg
        

38565


0

In [606]:
def row_op(row):

    if g_nx.nodes[row['src_node']]['name'] == g_nx.nodes[row['dest_node']]['name'] or g_nx.nodes[row['src_node']]['uri'] == g_nx.nodes[row['dest_node']]['uri']:
        return False
    return True

df_lessthan['is_valid'] = df_lessthan.apply(row_op,axis=1)
df_filter=df_lessthan[df_lessthan['is_valid']] 

In [549]:
def getId(row):
    return pd.Series([g_nx.nodes[row['src_node']]['name'],g_nx.nodes[row['src_node']]['uri'],g_nx.nodes[row['src_node']]['type'],g_nx.nodes[row['dest_node']]['name'],g_nx.nodes[row['dest_node']]['uri'],g_nx.nodes[row['dest_node']]['type']])
    
df_filter[["src_name","src_uri","src_type","dest_name","dest_uri","dest_type"]]=df_filter[df_lessthan['is_valid']].apply(getId,axis=1)

In [599]:
df_filter

preds  edge_label  src_node  dest_node  is_valid       src_name  \
362647  0.510759           0      1574       7400      True      Party mix   
362649  0.655622           0     31870      37070      True                  
362686  0.509911           0      1656       7428      True            😈😈😈   
362711  0.538797           0     30486      36983      True                  
362776  0.507164           0     10576       1109      True                  
...          ...         ...       ...        ...       ...            ...   
725119  0.577482           0      1989       5221      True      Michelle    
725157  0.604030           0      1259       2930      True  brad paisley    
725186  0.512304           0     36357       1149      True                  
725242  0.584185           0      1576       4524      True          sound   
725261  0.503086           0     15498       1605      True                  

                                     src_uri  src_type    dest_name  \
362647                                        playlist                
362649  spotify:track:6Rw3g4D3BCiipubFnmHocK     track         2016   
362686                                        playlist                
362711  spotify:track:0Yqkp4ScyWSKTlm7hpnDQN     track        Sleep   
362776  spotify:track:7Av3vXv9nv2pVT29QZX4B6     track  Brasileiras   
...                                      ...       ...          ...   
725119                                        playlist                
725157                                        playlist                
725186  spotify:track:5wpDupdKxD4aKNV8kbtdYt     track     Top hits   
725242                                        playlist                
725261  spotify:track:52ojopYMUzeNcudsoz7O9D     track      Country   

                                    dest_uri dest_type  
362647  spotify:track:7gUDfyfjTIcrFijLRvv6AX     track  
362649                                        playlist  
362686  spotify:track:6tg8tJPrrFuofADSiZPsWw     track  
362711                                        playlist  
362776                                        playlist  
...                                      ...       ...  
725119  spotify:track:2qF8Zpn7iJh9mk4RfwLbjM     track  
725157  spotify:track:0j6YwWZVHsIFKr5FDEu68o     track  
725186                                        playlist  
725242  spotify:track:53i9QgdZ2X0DdvqDOvt7r4     track  
725261                                        playlist  

[15364 rows x 11 columns]

In [603]:
df_final = df_filter[(df_filter['src_name']=="Party mix") | ( df_filter['dest_name']=="Party mix") ]
df_final.sort_values('preds',ascending=False)
uri_list=list(df_final.apply(lambda x: x['src_uri'] if x['src_uri']!='' else x['dest_uri'], axis=1 ))

In [553]:
!pip install spotipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.9/238.9 kB 5.7 MB/s eta 0:00:000:00:01
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)


In [573]:
uri_list=[]
uri_list = list(df_final["dest_uri"])

In [605]:
df_final

preds  edge_label  src_node  dest_node  is_valid   src_name  \
362647  0.510759           0      1574       7400      True  Party mix   
403861  0.632110           0     27010        575      True              
450506  0.509937           0      1574       7667      True  Party mix   
459937  0.502560           0     16597       1574      True              
460682  0.551944           0       575      29008      True  Party mix   
485977  0.510032           0      7638       1574      True              
494971  0.598414           0     27865        575      True              
498630  0.521360           0       575      29667      True  Party mix   
532552  0.544246           0       575      29159      True  Party mix   
566967  0.654191           0       575      26311      True  Party mix   
641748  0.509674           0      1574       8115      True  Party mix   
675285  0.686479           0     23626        575      True              
722548  0.538181           0       575      19498      True  Party mix   

                                     src_uri  src_type  dest_name  \
362647                                        playlist              
403861  spotify:track:50Pu9PTbnujupgl2LLwn5y     track  Party mix   
450506                                        playlist              
459937  spotify:track:705u4LfXcTBuDOB8UCy0A7     track  Party mix   
460682                                        playlist              
485977  spotify:track:13VU9JTEVLrDfFUi7ZhuA5     track  Party mix   
494971  spotify:track:3PfBUAunz826hdvH225T0K     track  Party mix   
498630                                        playlist              
532552                                        playlist              
566967                                        playlist              
641748                                        playlist              
675285  spotify:track:6WNZ22iJe51C0CfcT97GKC     track  Party mix   
722548                                        playlist              

                                    dest_uri dest_type  
362647  spotify:track:7gUDfyfjTIcrFijLRvv6AX     track  
403861                                        playlist  
450506  spotify:track:3uy0jtkM8QYVTsBazkli1x     track  
459937                                        playlist  
460682  spotify:track:2R6RRpbjQ9zSSzdqHMoisV     track  
485977                                        playlist  
494971                                        playlist  
498630  spotify:track:1GINsWBySdj1yI0vtjzlQz     track  
532552  spotify:track:7D3zhmFnxJSgI7cvkXan4l     track  
566967  spotify:track:2V8J29IIjB0SbwhvKMijrV     track  
641748  spotify:track:4Vxu50qVrQcycjRyJQaZLC     track  
675285                                        playlist  
722548  spotify:track:3l0pdat6N0XcotNr2b7rNl     track

In [575]:
uri_list

['spotify:track:7gUDfyfjTIcrFijLRvv6AX',
 'spotify:track:3uy0jtkM8QYVTsBazkli1x',
 'spotify:track:2R6RRpbjQ9zSSzdqHMoisV',
 'spotify:track:1GINsWBySdj1yI0vtjzlQz',
 'spotify:track:7D3zhmFnxJSgI7cvkXan4l',
 'spotify:track:2V8J29IIjB0SbwhvKMijrV',
 'spotify:track:4Vxu50qVrQcycjRyJQaZLC',
 'spotify:track:3l0pdat6N0XcotNr2b7rNl',
 'spotify:track:50Pu9PTbnujupgl2LLwn5y',
 'spotify:track:705u4LfXcTBuDOB8UCy0A7',
 'spotify:track:13VU9JTEVLrDfFUi7ZhuA5',
 'spotify:track:3PfBUAunz826hdvH225T0K',
 'spotify:track:6WNZ22iJe51C0CfcT97GKC']

In [604]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

def gettrackname(uri_list):
    # Replace the values below with your own Spotify API credentials
    client_id = 'd5566a60926740f3a8070889731a2d21'
    client_secret = 'eb5fc0638a1241c3a611186ff8d167e3'

    # Initialize the Spotify API client
    client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

    info = []
    for uri in uri_list:
        # Use the track method to get information about the track
        track_info = sp.track(uri)

        # Get the track name from the track information
        track_name = track_info['name']
        track_info = sp.track(uri)

        # Get the artist name from the track information
        artist_name = track_info['artists'][0]['name']
        info.append((track_name, artist_name))
        
    return info

print(gettrackname(uri_list))


[('Mamá Me Lo Contó', 'Gente De Zona'), ('Cómo te echo de menos', 'Alejandro Sanz'), ('Planet Rock', 'Afrika Bambaataa'), ('Kush & Corinthians', 'Kendrick Lamar'), ('Never Ever', 'Wiz Khalifa'), ('Satisfaction - Afrojack Remix', 'Benny Benassi'), ('Fix Me', 'Jasmine Thompson'), ('Willie, Waylon and Me', 'David Allan Coe'), ('Airborne', 'Running Cadence'), ('愛我請留言 - (劇集 "愛我請留言" 主題曲)', 'Jinny Ng'), ('Life Changes', 'Thomas Rhett'), ('Ms. Gravystone (feat. Mia Gladstone)', 'Yung Gravy'), ('A Violent Sky', 'Apparat')]
